In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

import os
import openai
from dotenv import load_dotenv, dotenv_values

load_dotenv()
config = dotenv_values(".env")

API_KEY = config["API_KEY"]
ENDPOINT = config["ENDPOINT"]

In [2]:
def client():
    try:
        ta_credential = AzureKeyCredential(API_KEY)
        text_analytics_client = TextAnalyticsClient(
            endpoint=ENDPOINT, 
            credential=ta_credential)
        return text_analytics_client
    except Exception as e:
        print("Error: {e}")
        return None

In [3]:
Documents = [
	"""
	I have to say that I love this thing. I read a majority of the reviews here on Amazon before I placed my order, and I can say that 95% of the complaints in other reviews are unwarranted.

    Complaints others have had:
    - Short/Fixed USB Cable :: I don't understand this one. Every single wired keyboard and mouse I've ever owned had a fixed cable of the same length. If it doesn't work for you, neither do your mouse and keyboard.
    - LCD "ripples" when you press buttons :: Try not pressing buttons like a 200lb gorilla. Mine doesn't ripple unl
    Complaints others have had:
    - Short/Fixed USB Cable :: I don't understand this one. Every single wired keyboard and mouse I've ever owned had a fixed cable of the same length. If it doesn't work for you, neither do your mouse and keyboard.
    - LCD "ripples" when you press buttons :: Try not pressing buttons like a 200lb gorilla. Mine doesn't ripple unless I really try to make it do so.
    - There's no built-in functions for MYSOFTWARE :: ... This one annoys me greatly. It's built for streaming software. It literally has STREAM in the name. That said, you can set up custom key binds with custom images for the keys all you want. They work great. I have mine doing a dozen functions that technically aren't supported for OBS, Photoshop, and several other programs.
    - It feels cheap/cheaply made :: What planet these people live on is beyond me... This thing is amazingly well built.
    - There's a long delay when I press buttons! :: Try not streaming on a potato. Mine works beautifully. Any delay is on the users end. (too much BS running in the background/slow computer)
    - It's overpriced! :: If you think this is overpriced, you haven't looked at similar products. Programmable keypads with similar functions (with NO LCD) are just as/more expensive.
    - There's only one LCD, it's not individual displays for each key! :: Who cares? It functions and looks great in the process.

    My Complaints:
    - Fairly minimal native OBS options. This is easily overcome with custom key binds, it would have just been nice to have things like mute mic built-in.
    - Always on display. It really needs to pick up on when the PC display shuts off due to inactivity and do the same. (or shut off when the computer is locked, either way)

    The TLDR: It's great. It works beautifully for streaming, and just as well as a general productivity helper. I don't regret ordering mine and I doubt anybody else will either. 
    """
]

In [4]:
client = client()
response = client.analyze_sentiment(
    documents=Documents, 
    language='en-US', 
    show_opinion_mining=True)

positive_reviews = [doc for doc in response if doc.sentiment == "positive"]
mixed_reviews = [doc for doc in response if doc.sentiment == "mixed"]
negative_reviews = [doc for doc in response if doc.sentiment == "negative"]

In [5]:
# Result 
# Predicted sentiment for document

for doc in response:
    print('Sentiment Analysis Outcome: {}'.format(doc.sentiment))   
    
    # Document level sentiment confidence scores between 0 and 1 for each sentiment label.
    print('Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n'.format(
        doc.confidence_scores.positive,
        doc.confidence_scores.neutral,
        doc.confidence_scores.negative,
    ))
    print('-'*75)
    

Sentiment Analysis Outcome: mixed
Overall scores: positive=0.45; neutral=0.13; negative=0.42 

---------------------------------------------------------------------------


In [7]:
# Break down the analysis by each sentence, then reference the sentence attribue to see the sentence text and score.
sentences = doc.sentences
for indx, sentence in enumerate(sentences):
    print('Sentence #{0}'.format(indx+1))
    print('Sentence text: {0}'.format(sentence.text))
    print('Sentence scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f}'.format(
        sentence.confidence_scores.positive,
        sentence.confidence_scores.neutral,
        sentence.confidence_scores.negative,
    ))
    
    # opinion analysit
    for mined_opinion in sentence.mined_opinions:
        target = mined_opinion.target
        print("...'{}' target '{}'".format(
            target.sentiment,
            target.text
        ))
        print("......Target score: \n......Positive={0:.2f}\n......Negative={1:.2f}\n".format(
            target.confidence_scores.positive,
            target.confidence_scores.negative,
        ))
        for assesment in mined_opinion.assessments:
            print("......'{}' assessment '{}'".format(
                assesment.sentiment,
                assesment.text
            ))
            print(".........Assessment score: \n.........Positive={0:.2f}\n.........Negative={1:.2f}\n".format(
                assesment.confidence_scores.positive,
                assesment.confidence_scores.negative,
            ))
    print()
    

Sentence #1
Sentence text:  	I have to say that I love this thing. 
Sentence scores: positive=0.93; neutral=0.05; negative=0.02

Sentence #2
Sentence text: I read a majority of the reviews here on Amazon before I placed my order, and I can say that 95% of the complaints in other reviews are unwarranted.

Sentence scores: positive=0.02; neutral=0.31; negative=0.68

Sentence #3
Sentence text:     Complaints others have had:     - Short/Fixed USB Cable :: I don't understand this one. 
Sentence scores: positive=0.01; neutral=0.28; negative=0.71

Sentence #4
Sentence text: Every single wired keyboard and mouse I've ever owned had a fixed cable of the same length. 
Sentence scores: positive=0.01; neutral=0.99; negative=0.01

Sentence #5
Sentence text: If it doesn't work for you, neither do your mouse and keyboard.     
Sentence scores: positive=0.02; neutral=0.53; negative=0.45

Sentence #6
Sentence text: - LCD "ripples" when you press buttons :: Try not pressing buttons like a 200lb gorilla